In [ ]:
%matplotlib inline
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import open3d as o3d
from pyntcloud import PyntCloud 
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN, OPTICS, SpectralCoclustering
from sklearn.metrics import pairwise_distances
import json
import os
from glob import glob
import random
import torch
import pygeodesic.geodesic as geodesic
import pyvista as pv
import vtk
from vtk_helpers import *
from scipy.spatial import Delaunay, ConvexHull
import potpourri3d as pp3d
from tqdm import tqdm
np.random.seed(0)

In [ ]:
root = "./Surface_Defects_pcd_extend_2000_geod_estnorm_noise0001/"
folders = ['normal']
N = 50
n = 2000
k = 10
sigma_list = []

for fold in folders:
      
    cate = root + fold + '/'

    for i in tqdm(range(N)):
        file = cate + fold + str(i+1) + '.txt'
        pts = np.loadtxt(file, delimiter = ',')[:,:3]
        dist_file = cate + fold + str(i+1) + '_geodesic_dis.npz'
        pair_dis = np.load(dist_file)['data']
        max_value = pair_dis.max()
        pair_dis[pair_dis<=0] = max_value
        knn_index = np.zeros((n,k+1), dtype = np.int32)
        knn_index[:,-1] = [i for i in range(n)]
        knn_index[:,:k] = np.argpartition(pair_dis, k)[:,:k]
        knn_graph = pts[knn_index]
        m1 = knn_graph - knn_graph.mean(axis = 1, keepdims = 1)
        cov = np.einsum('ikj,ikl->ijl',m1,m1) /(k)
        e,_ = LA.eig(cov)
        e.sort(axis=1)
        sigma = e[:,0] / e.sum(axis = 1)
        sigma_list.append(sigma)

sigma_arr = np.concatenate(sigma_list, axis = 0)            
normal_mean = sigma_arr.mean()
normal_std = sigma_arr.std()

In [ ]:
normal_mean

In [ ]:
root = "./Surface_Defects_pcd_extend_2000_geod_estnorm_noise0001/"
folders = ['raised', 'dent', 'crack']
N = 50
n = 2000
k = 10
thresh = normal_mean + 1.282 * normal_std ##80% CI
for fold in folders:
      
    cate = root + fold + '/'

    for i in tqdm(range(N)):
        #file = "./Surface_Defects_pcd_extend_2000_geod_estnorm/dent/dent1.txt"
        #dist_file = "./Surface_Defects_pcd_extend_2000_geod_estnorm/dent/dent1_geodesic_dis.npz"
        file = cate + fold + str(i+1) + '.txt'
        pts = np.loadtxt(file, delimiter = ',')[:,:3]
        dist_file = cate + fold + str(i+1) + '_geodesic_dis.npz'
        pair_dis = np.load(dist_file)['data']
        max_value = pair_dis.max()
        pair_dis[pair_dis<=0] = max_value
        knn_index = np.zeros((n,k+1), dtype = np.int32)
        knn_index[:,-1] = [i for i in range(n)]
        knn_index[:,:k] = np.argpartition(pair_dis, k)[:,:k]
        knn_graph = pts[knn_index]
        m1 = knn_graph - knn_graph.mean(axis = 1, keepdims = 1)
        cov = np.einsum('ikj,ikl->ijl',m1,m1) /(k)
        e,_ = LA.eig(cov)
        e.sort(axis=1)
        sigma = e[:,0] / e.sum(axis = 1)
        sigma[sigma>thresh] = 1
        sigma[sigma<=thresh] = 0
        #np.savez('./Surface_Defects_pcd_extend_2000_geod_estnorm/dent/dent1.npz', data = sigma)
        #break
        np.savez(cate + fold + str(i+1) + '.npz', data = sigma)